## 自动生成聊天的尝试

这个脚本由李鲁鲁开发， 属于[Chat凉宫春日](https://github.com/LC1332/Chat-Haruhi-Suzumiya) 

用于研究是否能够基于GPT生成大量的对话数据

**Chat凉宫春日**是模仿凉宫春日等一系列动漫人物，使用近似语气、个性和剧情聊天的语言模型，

<details>
  <summary> 由李鲁鲁，冷子昂，闫晨曦，封小洋等开发。 </summary>

李鲁鲁发起了项目，并完成了最早的版本，在多个微信群实现了测试。

冷子昂参与了早期Gradio的开发，并且参与了后端和前端的选型

闫晨曦将李鲁鲁的notebook重构为app.py

封小洋进行了中文转日文模型的选型

</details>

- [x] 对所有数据生成关键字
- [x] 对所有对话生成关键字
- [x] 尝试一下通过长对话反过来生成句子

我们来试验一下prompt

```
提取反引号文本中的关键字Entity，以list的形式输出在一个json中。

例子输入:`阿虚:「我无意中听到一件事。」
春日:「反正不会是什么重要的事。」`
例子输出:`{"Entity": ["不重要的事","阿虚","春日"]}`

例子输入:`阿虚:「你为什么要剪头发啊？」
春日:「没什么理由，就是想剪了而已。」`
例子输出:`{"Entity": ["剪头发","没什么理由"]}`

输入:`春日:「好想要一台电脑哦！」
旁白:春日盘着腿坐在一张从其他社团“拿”来的一张桌子，腿旁是一个小三角锥，上面用汉字写着大大的「团长」二字
春日:「在这个资讯化的时代里，连一台电脑都没有，是不行的！」
春日:「所以，我会弄一台电脑回来。」
阿虚:「弄一台，你是说电脑吗?去哪里弄?你该不会打算去抢电器行吧?」
春日:「怎么可能!是更近一点的地方啦!」
春日:「拿着这个拍立得。」
春日:「给我听好了!现在要告诉你作战计划，你可要按照计划行动喔!千万要好好把握机会。」
阿虚:「啊?你又要乱来啦?」
春日:「有什么关系！」`
```

ok这样很好，让我们来组织一下这个脚本

```
{"Entity": ["电脑", "资讯化的时代", "弄回来", "拍立得", "作战计划", "按照计划行动", "把握机会", "乱来"]}
```

这次我们考虑用LangChain来做

In [ ]:
#@title 安装环境
! pip -q install openai gradio transformers tiktoken langchain gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 101.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.4/969.4 kB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.1/288.1 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 19.6 MB/s eta 0:00

## 输入OpenAI Token



In [54]:
import os
import openai

openai.api_key = 'sk-lfrdoJK' # 在这里输入你的OpenAI API Token
os.environ["OPENAI_API_KEY"] = openai.api_key 

In [ ]:
KEYWORD_PROMPT = """
提取反引号文本中的关键字Entity，以list的形式输出在一个json中。
"""

In [ ]:
#@title 定义关键词提取prompt

from langchain.chat_models import ChatOpenAI

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chatModel = ChatOpenAI(temperature=0)

def extract_keywords( new_query ):

    input1 = """阿虚:「我无意中听到一件事。」
    春日:「反正不会是什么重要的事。」"""
    output1 = """{"Entity": ["不重要的事","阿虚","春日"]}"""

    input2 = """阿虚:「你为什么要剪头发啊？」
    春日:「没什么理由，就是想剪了而已。」"""
    output2 = """{"Entity": ["剪头发","没什么理由"]}"""

    messages = [
        SystemMessage(content=KEYWORD_PROMPT),
        HumanMessage(content=input1),
        AIMessage(content=output1),
        HumanMessage(content=input2),
        AIMessage(content=output2)
    ]

    messages.append(HumanMessage(content=new_query))

    return_msg = chatModel(messages)

    response = return_msg.content

    return response

In [ ]:
test_input = """旁白:春日从其中一个纸袋取出上头印了些手写文字的A4草稿纸。
春日:「这是为了让大家认识我们SOS团而特别做的传单。这两百张传单，可是我偷溜进印刷室辛辛苦苦印出来的喔!」
旁白:春日将传单分发给了大家。上面写着公告『SOS团创团声明 我们SOS团正大募集这世界上所有不可思议的事。欢迎过去曾经历不可思议事件的人，或是现在正面临不可思议、谜样现象的人，以及有预感不久的将来一定直经历奇幻事件的人踊跃与我们咨询。我们会尽力替你解决问题。不过，普通的不可思议事件恕不受理，一定要让我们觉得相当惊人的不可思议事件才行。敬请注意。电子信箱如下……』
春日:「好了，该去发传单了。」
阿虚:「去哪里发?」
春日:「校门口，现在还有很多学生没回家。」
阿虚:「是是是，你说的都对。」
春日:「你不用发没关系，实玖瑠跟我去就好了。」
阿虚:「什么?」
"""

keywords = extract_keywords(test_input)

print(keywords)

{"Entity": ["SOS团","传单","印刷室","大家","不可思议的事","不可思议事件","电子信箱","校门口","学生","实玖瑠"]}


## 将所有的经典故事，提取为关键字

In [ ]:
#@title 读取所有文本，存储到titles 和字典title_to_text

!git clone https://github.com/LC1332/Prophet-Andrew-Ng

prophet_data_folder = '/content/Prophet-Andrew-Ng/haruhi-data'

import os

import tiktoken

enc = tiktoken.get_encoding("cl100k_base")

titles = []
title_to_text = {}

# scan all txt file in prophet_data_folder
for file in os.listdir(prophet_data_folder):
    if file.endswith('.txt'):
        title_name = file[:-4]
        titles.append(title_name)

        with open(os.path.join(prophet_data_folder, file), 'r') as f:
            title_to_text[title_name] = f.read()

# report length of each text
for title in titles:
    print(title, len(enc.encode(title_to_text[title])),end= ' | ')

fatal: destination path 'Prophet-Andrew-Ng' already exists and is not an empty directory.
从哪儿搞电脑 319 | 电研社初次会面 416 | 介绍其他社员 254 | 日常3 216 | 地球上小小的螺丝钉 993 | 电子邮箱 143 | 最新的电脑 200 | 转学生 286 | 自我介绍 115 | 询问朝仓信息 362 | 初中交往经历 168 | 像普通人一样生活 684 | SOS团起名由来 265 | 朝仓转学 457 | 约翰史密斯 168 | 与朝仓公寓管理员谈话 474 | 兔女郎被老师驱散 444 | 开学第二天 210 | 交往的男生 638 | 社团教室 715 | 最后一名社员 357 | 古泉是男的还是女的 203 | 春日与阿虚 149 | 第一次全员大会 374 | 凉宫春日为何转变 154 | 让阿虚帮忙建社团 287 | 转学生的消息 236 | 奇怪的朝仓 296 | 搞电脑过程 438 | 自己建一个社团就好啦 353 | 无聊的社团 284 | 不重要的事情 38 | 电脑是怎么来的 153 | 拉壮丁 668 | 谁来写网站 193 | 团长设定 201 | 兔女郎的反应 239 | 颜色与星期 473 | 为什么剪头发 43 | 凉宫春日的基础设定 217 | 无聊的日常2 288 | 萌角色的重要性 692 | 春日与有希 101 | 没有灵异事件 665 | 兔女郎 332 | 带上阿虚去朝仓家 394 | 传单 424 | 找管理员借钥匙 115 | 

In [ ]:
keywords = []

for title in titles:
    text = title_to_text[title]
    keyword = extract_keywords(text)
    keywords.append(keyword)

In [ ]:
print(keywords[0])

{"Entity": ["电脑","资讯化的时代","拍立得","作战计划","把握机会"]}


In [ ]:
#@title 打开文件并写入jsonl

import json

with open("all_story_keywords.jsonl", "w", encoding='utf-8') as f:
    for keyword in keywords:
        try:
            keyword_dict = json.loads(keyword)
            json.dump(keyword_dict, f, ensure_ascii=False)
            f.write('\n')
        except:
            print("Error: cannot load keyword:", keyword)
            continue

这个非常令人满意，让我们把过往用户的对话也提取一下

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

save_path = "/content/drive/MyDrive/GPTData/Haruhi-Lulu/"

Mounted at /content/drive


In [ ]:
#@title 解析所有对话数据

import os
import json

save_path = "/content/drive/MyDrive/GPTData/Haruhi-Lulu/"

chat_data = []

for file_name in os.listdir(save_path):
    # print(file_name)
    file_path = os.path.join(save_path, file_name)
    with open(file_path, 'r', encoding='utf-8') as f:
        data = f.read()
        segments = data.split('\n---\n')
        for i in range(0, len(segments)-1, 2):
            query = segments[i]
            response = segments[i+1]
            role_a = role_b = ""
            if ':' in query:
                role_a, query = query.split(':', 1)
            elif '：' in query:
                role_a, query = query.split('：', 1)
            if ':' in response:
                role_b, response = response.split(':', 1)
            elif '：' in response:
                role_b, response = response.split('：', 1)
            chat = {"role_A":role_a.strip(), "role_B":role_b.strip(), "query":query.strip(), "response":response.strip()}
            chat_data.append(chat)

print(json.dumps(chat_data, ensure_ascii=False))

cleaned_chat_data = []

for i in range(len(chat_data)):
    is_duplicate = False
    for j in range(i+1, len(chat_data)):
        if chat_data[i]['query'] == chat_data[j]['query'] and chat_data[i]['response'] == chat_data[j]['response']:
            is_duplicate = True
            break
    if not is_duplicate:
        cleaned_chat_data.append(chat_data[i])

chat_datas = cleaned_chat_data

In [ ]:
print(len(chat_datas))

560


In [ ]:
chat_datas[0]['key'] = ['new_key']
print(chat_datas[0])

{'role_A': '阿虚', 'role_B': '春日', 'query': '「今天在计算机课上老师教了我写Python!」', 'response': '「哦？Python？那你能不能帮我写一个程序啊？」', 'key': ['new_key']}


In [87]:
def json_chat_2_str( data ):
    return str(data['role_A'] + ':' + data['query'] + '\n' + data['role_B'] + ':' + data['response'])

def json_chat_2_str_A( data ):
    return str(data['role_A'] + ':' + data['query'])

In [88]:
from tqdm import tqdm

for chat in tqdm(chat_datas, desc="Extracting Keywords"):
    text = json_chat_2_str_A(chat)
    response = extract_keywords(text)
    chat['key_json'] = response


Extracting Keywords: 100%|██████████| 560/560 [26:20<00:00,  2.82s/it]


In [95]:
#@title 保存chat_dats为pkl

import pickle
fname = '/content/drive/MyDrive/GPTData/haruhi_chat_datas.pkl'
with open(fname, 'wb') as f:
    pickle.dump(chat_datas, f)

def load_chat_datas():
    with open(fname, 'rb') as f:
        chat_datas = pickle.load(f)

    return chat_datas

In [96]:
print(chat_datas[1]['key_json'])

{"Entity": ["写程序","什么样的程序"]}


In [97]:
print(chat_datas[1].keys())

dict_keys(['role_A', 'role_B', 'query', 'response', 'key_json'])


In [98]:
print(chat_datas[1])

{'role_A': '阿虚', 'role_B': '春日', 'query': '「你想写一个什么样的程序呢？」', 'response': '「我想写一个能够预测未来的程序，可以预测天气、地震、彩票号码等等。」', 'key_json': '{"Entity": ["写程序","什么样的程序"]}'}


chat_datas是一个python的list，每个元素的keys为'role_A', 'role_B', 'query', 'response', 'key_json'

其中key_json是一个类似的json字符串

{"Entity": ["写程序","预测未来","天气","地震","彩票号码"]}

- 遍历chat_datas中的每一个元素，尝试解析key_json字段，如果解析成功，则把其中的Entity存成keyword，不成功则跳过
- 对每一个元素，只保存'role_A', 'role_B', 'query', 'response', 'keywords'字段，其他字段都忽略

样例element输入:
`{'role_A': '阿虚', 'role_B': '春日', 'query': '「你想写一个什么样的程序呢？」', 'response': '「我想写一个能够预测未来的程序，可以预测天气、地震、彩票号码等等。」','other':'to remove',  'key_json': '{"Entity": ["写程序","预测未来","天气","地震","彩票号码"]}`
样例element输出:
`{'role_A': '阿虚', 'role_B': '春日', 'query': '「你想写一个什么样的程序呢？」', 'response': '「我想写一个能够预测未来的程序，可以预测天气、地震、彩票号码等等。」', 'keywords': ["写程序","预测未来","天气","地震","彩票号码"]}`

并把chat_datas中的信息，以jsonl格式保存到all_chat_datas.jsonl

我们还是争取写个jsonl出来

In [99]:
import json

all_chat_datas = []
for element in chat_datas:
    try:
        key_json = json.loads(element['key_json'])
        keywords = key_json['Entity']
    except:
        print('failed to parse', element['key_json'])
        continue
    new_element = {
        'role_A': element['role_A'],
        'role_B': element['role_B'],
        'query': element['query'],
        'response': element['response'],
        'keywords': keywords
    }
    all_chat_datas.append(new_element)

with open('all_chat_datas.jsonl', 'w', encoding='utf-8') as f:
    for element in all_chat_datas:
        json.dump(element, f, ensure_ascii=False) 
        f.write('\n')

failed to parse 抱歉，我不理解您的意思。请您提供更多上下文信息或者明确您的问题。
failed to parse 无法提取关键字Entity，因为这句话中没有明确的实体或关键词。
failed to parse 无法提取关键字Entity，因为反引号文本中没有明显的实体或关键词。
failed to parse {"Entity": []}（无关键字）


In [100]:
print(chat_datas[12]['key_json'])

{"Entity": ["李鲁鲁","Haruhi","新转学生","同学"]}


让我们来尝试生成这个prompt

根据keywords的内容补全text，text为对于凉宫春日剧情的一些讨论问题，用一致性的语言风格，根据每行中的json内容，根据keywords中的关键字，补全text的内容。

输入:
```
{"keywords":["社团","不想来"]}
{"keywords":["新来","转学","转学生"]}
{"keywords":["长门","外星人"]}
{"keywords":["社团","成员","长门","古泉","学姐"]}
```
输出:
```
{"role":"阿虚", "text":"明天我有点不想来社团活动了", "keywords":["社团","不想来"]}
{"role":"李鲁鲁", "text":"你好呀，Haruhi， 我是新转学来的同学李鲁鲁", "keywords":["新来","转学","转学生"]}
{"role":"李鲁鲁", "text":"什么，你知道长门有希是外星人？", "keywords":["长门","外星人"]}
{"role":"阿虚", "text":"不会吧，怎么只有我和你呢？长门、古一和学姐不是我们社团的成员吗？", "keywords":["社团","成员","长门","古泉","学姐"]}
```

输入:
```
{"keywords":["店员", "一览表", "名单", "相机"]}
{"keywords":[ "转学", "国外", "灵异事件"]}
{"keywords":["长门","外星人"]}
{"keywords":["社团","成员","长门","古泉","学姐"]}
```

这个效果还是不错的，GPT的输出为

```
{"role":"某个顾客", "text":"店员，你能不能给我一份这个店里的一览表？我想看看名单", "keywords":["店员", "一览表", "名单"]}
{"role":"古泉一树", "text":"大家好，我是新转学来的同学古泉一树，来自国外", "keywords":[ "转学", "国外"]}
{"role":"长门有希", "text":"其实我是外星人", "keywords":["长门","外星人"]}
{"role":"阿虚", "text":"不会吧，怎么只有我和你呢？长门、古一和学姐不是我们社团的成员吗？", "keywords":["社团","成员","长门","古泉","学姐"]}
```

那我们后面要做的就是给关键词去重就可以

我们现在要做的是，从chat中选取10组关键字，组成few shot

In [101]:
print(all_chat_datas[0].keys())
print(all_chat_datas[0])

dict_keys(['role_A', 'role_B', 'query', 'response', 'keywords'])
{'role_A': '阿虚', 'role_B': '春日', 'query': '「今天在计算机课上老师教了我写Python!」', 'response': '「哦？Python？那你能不能帮我写一个程序啊？」', 'keywords': ['计算机课', 'Python']}


sel_chat_datas是一个python的list of dict，长度为n

每个元素的数据格式如下

`{'role_A': '阿虚', 'role_B': '春日', 'query': '「今天在计算机课上老师教了我写Python!」', 'response': '「哦？Python？那你能不能帮我写一个程序啊？」', 'keywords': ['计算机课', 'Python', '写程序', '阿虚', '春日']}`

我们希望实现一个函数，输入为sel_chat_datas，输出为sample_input, sample_output

其中第一个输出 sample_input为n行，每行只保留json中的keywords

每行的格式如下

`{'keywords': ['计算机课', 'Python', '写程序']}`

第二个输出 sample_outout为n行，将role_A重命名为role，query重命名为text，并且保留keywords，每行的格式如下

`{'role': '阿虚', 'text': '「今天在计算机课上老师教了我写Python!」', 'keywords': ['计算机课', 'Python', '写程序']}`

- 注意这其中每行的keywords中去掉了stop_words中的元素，stop_words = ['春日','阿虚','凉宫','凉宫春日']

第三个输出为一个list，包含了去重之后的keywords中的元素

请用python实现这个函数

In [102]:
print(len(all_chat_datas))

556


In [104]:
stop_words = ['春日','阿虚','凉宫','凉宫春日']


import random 

n = len(all_chat_datas)
sel_all = random.sample(range(1, n+1), 20)
sel_ids = sel_all[0:10]
sel_test = sel_all[10:]

sel_chat_datas = [ all_chat_datas[i] for i in sel_ids ]

from typing import List, Dict, Tuple

def process_sel_chat_data(sel_chat_datas: List[Dict[str, str]]) -> Tuple[List[Dict], List[Dict], List[str]]:
    stop_words = ['春日', '阿虚', '凉宫', '凉宫春日']
    sample_input = []
    sample_output = []
    all_keywords = set()
    for element in sel_chat_datas:
        keywords = [kw for kw in element['keywords'] if kw not in stop_words]
        np.random.shuffle(keywords)
        sample_input.append({'keywords': keywords})
        output_element = {
            'keywords': keywords,
            'role': element['role_A'],
            'text': element['query'],
        }
        sample_output.append(output_element)
        for kw in keywords:
            all_keywords.add(kw)
    return sample_input, sample_output, list(all_keywords)


sample_input, sample_output, sample_keywords = process_sel_chat_data( sel_chat_datas )

sel_test = [ all_chat_datas[i] for i in sel_test ]

test_input, _, _ = process_sel_chat_data( sel_test )


for sam in sample_input:
    print(sam)
# print(sample_input,'\n')

print('\n')

for sam in sample_output:
    print(sam)

print('\n')

for sam in test_input:
    print(sam)

print('\n')

print(sample_keywords)

{'keywords': ['如何评价']}
{'keywords': ['超能力', '轻功', '御剑飞行']}
{'keywords': ['古墓', '进入']}
{'keywords': []}
{'keywords': ['月球流浪', '求婚']}
{'keywords': ['明天', '河边', 'gerry']}
{'keywords': ['kiki', '对呀']}
{'keywords': ['校园第一节minecraft搭建大赛']}
{'keywords': ['破解案子', '投资人', '柯南', '5000亿日元']}
{'keywords': ['SOS团', '活动']}


{'keywords': ['如何评价'], 'role': '将', 'text': '「如何评价阿虚」'}
{'keywords': ['超能力', '轻功', '御剑飞行'], 'role': '阿虚', 'text': '「如果他真的会御剑飞行，那算是一种轻功，还算是一种超能力呢？」'}
{'keywords': ['古墓', '进入'], 'role': '柯南', 'text': '「进入古墓」'}
{'keywords': [], 'role': 'cj', 'text': '「我也是这样想的！」'}
{'keywords': ['月球流浪', '求婚'], 'role': '名人漩涡', 'text': '「那你还愿意和我一起到月球流浪么？还愿意向我求婚么？」'}
{'keywords': ['明天', '河边', 'gerry'], 'role': 'gerry', 'text': '「明天去河边」'}
{'keywords': ['kiki', '对呀'], 'role': 'kiki', 'text': '「对呀」'}
{'keywords': ['校园第一节minecraft搭建大赛'], 'role': '阿虚', 'text': '「我们是不是可以举办校园第一节minecraft搭建大赛」'}
{'keywords': ['破解案子', '投资人', '柯南', '5000亿日元'], 'role': '柯南', 'text': '「我这个月破解了好几个案子, 碰到了一个投资人,说要给我们5000亿日元, 我们接受吗」'}
{

根据keywords的内容补全text，text为对于凉宫春日剧情的一些讨论问题，用一致性的语言风格，根据每行中的json内容，根据keywords中的关键字，补全text的内容。

输入:
```js
{'keywords': ['冷子昂', '你好', '有什么事吗']}
{'keywords': ['班里', '亚丝娜', '同学', '个性', '女孩子', '好朋友', '光线照射', '情况', '线索', '证据', '调查']}
{'keywords': ['miku', '你好', '谁', '见过']}
{'keywords': ['英文名', 'Cosmic Exploration Club', '缩写', 'CEC', '国际化', '记忆']}
{'keywords': ['123', '奇怪的话']}
{'keywords': ['不可能', '弟弟', '搞错了人', '宫热']}
{'keywords': ['普通人', '高中生', '超能力', '神秘身份', '未知领域', '科学', '理性', '神秘', '超自然', '问题', '疑虑', '交流', '互相猜疑', '攻击']}
{'keywords': ['羽毛球', '队友关系', '任务', '个人感情', '工作', '社团活动', '电影']}
{'keywords': ['长门有希', '外星人', '超能力', '思维方式', '行为方式', '纯真', '善良', 'SOS团', '有趣', '神秘']}
{'keywords': ['约翰史密斯', '名字']}
```

输出:
```js
{'keywords': ['如何评价'], 'role': '将', 'text': '「如何评价阿虚」'}
{'keywords': ['超能力', '轻功', '御剑飞行'], 'role': '阿虚', 'text': '「如果他真的会御剑飞行，那算是一种轻功，还算是一种超能力呢？」'}
{'keywords': ['古墓', '进入'], 'role': '柯南', 'text': '「进入古墓」'}
{'keywords': [], 'role': 'cj', 'text': '「我也是这样想的！」'}
{'keywords': ['月球流浪', '求婚'], 'role': '名人漩涡', 'text': '「那你还愿意和我一起到月球流浪么？还愿意向我求婚么？」'}
{'keywords': ['明天', '河边', 'gerry'], 'role': 'gerry', 'text': '「明天去河边」'}
{'keywords': ['kiki', '对呀'], 'role': 'kiki', 'text': '「对呀」'}
{'keywords': ['校园第一节minecraft搭建大赛'], 'role': '阿虚', 'text': '「我们是不是可以举办校园第一节minecraft搭建大赛」'}
{'keywords': ['破解案子', '投资人', '柯南', '5000亿日元'], 'role': '柯南', 'text': '「我这个月破解了好几个案子, 碰到了一个投资人,说要给我们5000亿日元, 我们接受吗」'}
{'keywords': ['SOS团', '活动'], 'role': 'Kyon', 'text': '「最近SOS团有什么活动吗」'}
```

输入:
```js
{'keywords': ['看法']}
{'keywords': ['谁']}
{'keywords': ['Kyon']}
{'keywords': ['求解', '常微分非齐次线性方程']}
{'keywords': ['吉他水平', '长门有希']}
{'keywords': ['打羽毛球', '每天晚上', '运动天赋', '工作']}
{'keywords': ['下课']}
{'keywords': ['你', '还好']}
{'keywords': ['上次和朝比奈一起穿兔女郎', '特别的社团活动']}
{'keywords': ['永葆青春', '奇异的药水', '艰辛', '奇怪的感情', '永生之路', '帮助', '古墓', '改变基因']}
```

GPT的输出:
```js
{'keywords': ['看法'], 'role': 'cj', 'text': '「你们对这个问题有什么看法吗？」'}
{'keywords': ['谁'], 'role': '阿虚', 'text': '「你说的是谁？」'}
{'keywords': ['Kyon'], 'role': '长门有希', 'text': '「Kyon，你想说什么？」'}
{'keywords': ['求解', '常微分非齐次线性方程'], 'role': '小兰', 'text': '「求解那个常微分非齐次线性方程还是挺难的呢」'}
{'keywords': ['吉他水平', '长门有希'], 'role': '长门有希', 'text': '「你们知道吗，长门有希的吉他水平其实非常高！」'}
{'keywords': ['打羽毛球', '每天晚上', '运动天赋', '工作'], 'role': '小美', 'text': '「我每天晚上都要打羽毛球，但是我又有很多工作要做，真的是很累啊！不过我相信我自己的运动天赋，会一直保持好的状态的！」'}
{'keywords': ['下课'], 'role': 'cj', 'text': '「好了，下课了！」'}
{'keywords': ['你', '还好'], 'role': '阿虚', 'text': '「你觉得我还好吗？」'}
{'keywords': ['上次和朝比奈一起穿兔女郎', '特别的社团活动'], 'role': '小兰', 'text': '「上次和朝比奈一起穿兔女郎的时候，真的是特别的社团活动啊」'}
{'keywords': ['永葆青春', '奇异的药水', '艰辛', '奇怪的感情', '永生之路', '帮助', '古墓', '改变基因'], 'role': '柯南', 'text': '「永葆青春、永生之路，这是一场充满艰辛和奇怪感情的旅程，但是只要我们帮助改变基因就可以了，去探索那个古墓吧！」'}
```

这个notebook有点乱了，因为我们已经把关键词都保存到了两个jsonl中

all_chat_datas.jsonl和 all_story_keywords.jsonl

让我们重启一个notebook重新开始